In [ ]:
import matplotlib.pyplot as plt
import mlflow
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from datetime import datetime
from mlflow.tracking import MlflowClient
from PIL import Image
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from tqdm import tqdm

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("mnist-classifier")

In [ ]:
def get_data_loaders():

    print("Creating data loaders")

    BATCH_SIZE = 1024

    dataloader_kwargs = {
        'batch_size': BATCH_SIZE,
        'num_workers': 1,
        'persistent_workers': True,
        'pin_memory': True,
        'shuffle': True
    }

    data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

    train_dataset_full = datasets.MNIST(root='./data', train=True, download=True, transform=data_transform)

    train_size = int(0.85 * len(train_dataset_full))
    val_size = len(train_dataset_full) - train_size

    seed = 42
    train_dataset, val_dataset = torch.utils.data.random_split(
        train_dataset_full, [train_size, val_size], generator=torch.Generator().manual_seed(seed)
    )

    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=data_transform)

    dataloaders = {}
    dataloaders["train"] = DataLoader(train_dataset,**dataloader_kwargs)
    dataloaders["val"] = DataLoader(val_dataset, **dataloader_kwargs)
    dataloaders["test"] = DataLoader(test_dataset, **dataloader_kwargs)

    print("Data loaders created successfully")

    return dataloaders

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
def train(model, device, dataloaders, num_epochs):

    print("\nStarting training...")

    learning_rate = 1.0
    optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

    mlflow.log_param("optimizer", "Adadelta")
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("scheduler_step_size", 1)
    mlflow.log_param("scheduler_gamma", 0.7)
    mlflow.log_param("epochs", num_epochs)
    mlflow.log_param("batch_size", dataloaders["train"].batch_size)

    for epoch in range(num_epochs):
        print()
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_correct = 0

            # Wrap dataloader with tqdm for progress bar
            loop = tqdm(dataloaders[phase], desc=f"{phase.capitalize()} Phase", leave=False)

            for inputs, labels in loop:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    loss = F.nll_loss(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_correct += torch.sum(preds == labels)

                # Update tqdm description dynamically
                loop.set_postfix({
                    "Loss": f"{loss.item():.4f}",
                    "Batch Acc": f"{(preds == labels).float().mean().item():.4f}"
                })

            if phase == "train":
                scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_correct.double() / len(dataloaders[phase].dataset)

            mlflow.log_metric(f"{phase}_loss", epoch_loss, step=epoch)
            mlflow.log_metric(f"{phase}_acc", epoch_acc, step=epoch)

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
            
    os.makedirs("model", exist_ok=True)
    torch.save(model.state_dict(), "model/mnist_cnn.pt")
    mlflow.pytorch.log_model(model, "model", registered_model_name="MNISTClassifier")
    print("\nTraining complete.")
            

In [ ]:
def log_confusion_matrix(model, dataloader, device):
    all_preds = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            preds = outputs.argmax(dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues')

    os.makedirs("artifacts", exist_ok=True)
    cm_path = f"artifacts/confusion_matrix.png"
    plt.savefig(cm_path)
    plt.close()

    mlflow.log_artifact(cm_path, artifact_path="plots")


In [ ]:
def log_misclassified_images(model, dataloader, device, max_images=25):
    model.eval()
    errors = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            mismatches = preds != labels

            for i in range(inputs.size(0)):
                if mismatches[i]:
                    errors.append((inputs[i].cpu(), preds[i].item(), labels[i].item()))
                if len(errors) >= max_images:
                    break
            if len(errors) >= max_images:
                break

    if errors:
        fig, axes = plt.subplots(5, 5, figsize=(10, 10))
        for ax, (img, pred, true) in zip(axes.flat, errors):
            ax.imshow(img.squeeze(), cmap="gray")
            ax.set_title(f"P: {pred} / T: {true}")
            ax.axis('off')
        plt.tight_layout()

        img_path = "artifacts/misclassified.png"
        plt.savefig(img_path)
        plt.close()

        mlflow.log_artifact(img_path, artifact_path="plots")


In [ ]:
def test(model, device, test_loader):
    print("\nStarting testing...")

    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            test_loss += F.nll_loss(outputs, labels, reduction='sum').item()
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels)

    test_loss /= len(test_loader.dataset)
    test_acc = correct.double() / len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * test_acc))
    
    mlflow.log_metric(f"test_loss", test_loss)
    mlflow.log_metric(f"test_acc", test_acc)
    
    log_confusion_matrix(model, test_loader, device)

    log_misclassified_images(model, test_loader, device)



In [ ]:
def training_workflow():

    with mlflow.start_run(run_name=f"mnist_run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):

        MANUAL_SEED = 42
        torch.manual_seed(MANUAL_SEED)
        mlflow.log_param("manual_seed", MANUAL_SEED)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f'Device: {device}\n')

        mlflow.set_tag("model_type", "LeNet-5")
        mlflow.set_tag("data_scientist", "Facundo Galan")
        mlflow.set_tag("dataset", "MNIST")
        mlflow.set_tag("framework", "PyTorch")
        mlflow.set_tag("device", device)

        dataloaders = get_data_loaders()

        model = Net().to(device)

        train(model, device, dataloaders, num_epochs=5)
        
        test(model, device, dataloaders["test"])

In [ ]:
training_workflow()

In [ ]:
client = MlflowClient()
model_name = "MNISTClassifier"

latest_versions = client.get_latest_versions(model_name, stages=["None"])
model_version = latest_versions[0].version

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage="Staging",  # También puede ser "Production", "Archived"
)

client.set_model_version_tag(model_name, model_version, "comment", "First stable version of MNIST classification model.")
client.set_model_version_tag(model_name, model_version, "custom_version", "v1.0.0")


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

image_path = "./three_example.png"
image = Image.open(image_path)

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
input_tensor = transform(image).unsqueeze(0).to(device)

model_uri = "models:/MNISTClassifier/Staging"
model = mlflow.pytorch.load_model(model_uri).to(device)
model.eval()

with torch.no_grad():
    output = model(input_tensor)
    probabilities = torch.exp(output)
    pred_label = torch.argmax(probabilities, dim=1).item()
    confidence = torch.max(probabilities).item()

plt.imshow(image, cmap="gray")
plt.title(f"Predicción: {pred_label} (Confianza: {confidence:.2%})")
plt.axis("off")
plt.show()